<a href="https://colab.research.google.com/github/varunsawhney8/Projects/blob/main/Final_draft%20first%20copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import pandas as pd
import numpy as np


path='/content/Reliance_Final_Ndtv_data4 (1).xlsx'
data=pd.read_excel(path)


data=data[::-1].reset_index()
data.drop(columns=['index'],axis=True,inplace=True)

path='/content/Reliance_Nse_data2.csv'
data1=pd.read_csv(path)


data.drop(columns=['Media','Article','Keywords'],inplace=True,axis=1)

In [49]:
data.columns

Index(['Date', 'Title'], dtype='object')

In [75]:
!pip install transformers
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model="ProsusAI/finbert")
list3=[]

for i in range(len(data['Title'])):
    a=(classifier(data['Title'][i]))
    a=a[0]
    a=list(a.values())[0]
    label_f={'positive':1,'negative':2,'neutral':0}
    list3.append(label_f[a])

data['1day_sentiment']=list3

del list3

In [76]:
aggregation_functions = {'Title': 'first', '1day_sentiment': 'max'}
data = data.groupby(data['Date']).aggregate(aggregation_functions)
data.reset_index( inplace=True)
data['Date']=pd.to_datetime(data['Date']).dt.date
data['1day_sentiment']=data['1day_sentiment'].astype('float')

In [77]:
data['3day_sentiment'] = round(data['1day_sentiment'].ewm(span=3).mean())
data['7day_sentiment'] = round(data['1day_sentiment'].ewm(span=7).mean())
data['15day_sentiment'] = round(data['1day_sentiment'].ewm(span=15).mean())
data.head()

,Date,Title,1day_sentiment,3day_sentiment,7day_sentiment,15day_sentiment
0,2020-07-16,Reliance's Stake Sale Talks With Saudi Aramco ...,2.0,2.0,2.0,2.0
1,2020-07-17,Reliance To Shut Crude Refining Unit For Maint...,2.0,2.0,2.0,2.0
2,2020-07-20,Reliance-Google Smartphone Deal in India Threa...,2.0,2.0,2.0,2.0
3,2020-07-21,"Netflix, Amazon, Facebook Stocks Draw In Amate...",1.0,1.0,2.0,2.0
4,2020-07-22,Reliance Industries Postpones June Quarter Res...,2.0,2.0,2.0,2.0


In [46]:
data.columns

Index(['Date', 'Title', 'finbert', '3day_sentiment', '7day_sentiment',
       '15day_sentiment'],
      dtype='object')

In [78]:
data1['Date']=pd.to_datetime(data1['Date']).dt.date
data1['perc_change1'] =data1['Close'].pct_change(periods=1)*100
#data1['perc_change3'] =data1['Close'].pct_change(periods=2)*100
#data1['perc_change7'] =data1['Close'].pct_change(periods=6)*100
data1.head()

,Date,Close,perc_change1
0,2020-07-16,1843.400024,NaN
1,2020-07-17,1911.699951,3.705106
2,2020-07-20,1919.900024,0.428941
3,2020-07-21,1971.550049,2.690246
4,2020-07-22,2004.000000,1.645911


In [84]:
data2.columns

Index(['Date', '1day_sentiment', '3day_sentiment', '7day_sentiment',
       '15day_sentiment', 'perc_change1'],
      dtype='object')

In [85]:
data2=data.drop(columns=['Title'],axis=1)
data2["perc_change1"]=[np.nan for i in range(len(data))]
label={0: 'Neutral',1:'Positive',2:'Negative'}
data2['1day_sentiment']=[label[data2['1day_sentiment'][i]]for i in range(len(data2))]
data2['3day_sentiment']=[label[data2['3day_sentiment'][i]]for i in range(len(data2))]
data2['7day_sentiment']=[label[data2['7day_sentiment'][i]]for i in range(len(data2))]
data2['15day_sentiment']=[label[data2['15day_sentiment'][i]]for i in range(len(data2))]

In [86]:
for i in range(len(data2)):
  for j in range(len(data1)):
    if data['Date'][i]==data1['Date'][j]:
      data2['perc_change1'][i]=data1['perc_change1'][j+1]
  count=0
  if type(data2['perc_change1'][i]) and pd.isna(data2['perc_change1'][i]):
    for j in range(len(data1)):
      if data2['Date'][i]>data1['Date'][j]:
        count+=1
    data2['perc_change1'][i]=data1['perc_change1'][count]

data2['perc_change1'][0]=np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] ==

In [97]:
data2['Close']=[np.nan for i in range(len(data))]

In [98]:
for i in range(len(data2)):
  for j in range(len(data1)):
    if data['Date'][i]==data1['Date'][j]:
      data2['Close'][i]=data1['Close'][j+1]
  count=0
  if type(data2['Close'][i]) and pd.isna(data2['Close'][i]):
    for j in range(len(data1)):
      if data2['Date'][i]>data1['Date'][j]:
        count+=1
    data2['Close'][i]=data1['Close'][count]



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [99]:
data2.head()

,Date,1day_sentiment,3day_sentiment,7day_sentiment,15day_sentiment,perc_change1,Close
0,2020-07-16,Negative,Negative,Negative,Negative,NaN,1911.699951
1,2020-07-17,Negative,Negative,Negative,Negative,0.428941,1919.900024
2,2020-07-20,Negative,Negative,Negative,Negative,2.690246,1971.550049
3,2020-07-21,Positive,Positive,Negative,Negative,1.645911,2004.000000
4,2020-07-22,Negative,Negative,Negative,Negative,2.684633,2057.800049


In [89]:
data2.columns

Index(['Date', '1day_sentiment', '3day_sentiment', '7day_sentiment',
       '15day_sentiment', 'perc_change1'],
      dtype='object')

In [105]:
data4=pd.DataFrame(columns=["Description","Total Opportunities","Profit(%)","Loss(%)","No Profit No Loss(%)"])
desc=[]
total1=[]
profit1=[]
loss1=[]
npnl1=[]
col=['1day_sentiment', '3day_sentiment', '7day_sentiment','15day_sentiment']
for j in col:
  profit=0
  loss=0
  npnl=0
  for i in range(len(data2)):
      if data2[j][i]=='Negative' and  data2['perc_change1'][i]<0:
        profit+=1
      elif data2[j][i]=='Negative' and  data2['perc_change1'][i]>0:
        loss+=1
        
      if data2[j][i]=='Positive' and  data2['perc_change1'][i]>0:
        profit+=1
      elif data2[j][i]=='Positive' and  data2['perc_change1'][i]<0:
        loss+=1
      if data2[j][i]=='Neutral':
        npnl+=1
  total= profit + loss +npnl
  desc.append(j)
  total1.append(total)
  profit1.append(profit/total*100)
  loss1.append(loss/total*100)
  npnl1.append(npnl/total*100)

data4["Description"]=["Rolling 1 day-News Day Event","Rolling 3 day-News Day Event","Rolling 7 day-News Day Event ","Rolling 15 day-News Day Event"]
data4["Total Opportunities"]=total
data4['Profit(%)']=profit1
data4['Loss(%)']=loss1
data4['No Profit No Loss(%)']=npnl1
print("Back Testing News Based Investment Oppurtunities")
print("Square off trade within  next trading day ")
display(data4)

Back Testing News Based Investment Oppurtunities
Square off trade within  next trading day 


,Description,Total Opportunities,Profit(%),Loss(%),No Profit No Loss(%)
0,Rolling 1 day-News Day Event,174,44.252874,35.057471,20.689655
1,Rolling 3 day-News Day Event,174,47.126437,43.678161,9.195402
2,Rolling 7 day-News Day Event,174,45.402299,53.448276,1.149425
3,Rolling 15 day-News Day Event,174,44.827586,55.172414,0.000000


In [45]:
data_1=data.drop(columns=["Title"],axis=1)
data_f1=data_1.iloc[len(data_1)-1,]
label={0.0: 'Neutral',1.0:'Positive',2.0:'Negative'}
f_data=pd.DataFrame(columns=["Description", "Sentiment"])
f_data["Description"]=["Rolling 1 day-News Day Event","Rolling 3 day-News Day Event","Rolling 7 day-News Day Event ","Rolling 15 day-News Day Event"]
f_data["Sentiment"]=[label[data_f1[i]] for i in range(1,len(data_f1),1)]

print("Current News Based Sentiment Report")
print("Last News Day Recoded : ",data_f1[0])
del data_1,data_f1
from IPython.display import display
display(f_data)


Current News Based Sentiment Report
Last News Day Recoded :  2021-08-16


,Description,Sentiment
0,Rolling 1 day-News Day Event,Positive
1,Rolling 3 day-News Day Event,Positive
2,Rolling 7 day-News Day Event,Positive
3,Rolling 15 day-News Day Event,Positive
